In [8]:
from utils import *


data = load_from_HDF("../data/LINCS2020/data_example/processed_data_id.h5")
print(data.keys())
print(data['LINCS_index'][1])


dict_keys(['LINCS_index', 'canonical_smiles', 'cid', 'sig'])
1


In [13]:
import pickle
import pandas as pd

# 读基因信息
df_gene = pd.read_csv('../data/LINCS2020/geneinfo_processed.csv')

# landmark 基因
df_landmark_gene = df_gene[(df_gene['pr_is_bing'] == 1) & (df_gene['pr_is_lm'] == 1)]
landmark_ids = df_landmark_gene['pr_id'].tolist()
landmark_symbols = df_landmark_gene['pr_gene_symbol'].tolist()

# best-infer 基因
df_best_infer_gene = df_gene[(df_gene['pr_is_bing'] == 1) & (df_gene['pr_is_lm'] == 0)]
best_infer_ids = df_best_infer_gene['pr_id'].tolist()
best_infer_symbols = df_best_infer_gene['pr_gene_symbol'].tolist()

# 打开表达矩阵
with open('../data/LINCS2020/modz_x1.pickle', 'rb') as f:
    dict_modz_x1_all_cid = pickle.load(f)

selected_cid = "A375"  # 换成你实际跑的 args.cell
x1_array = dict_modz_x1_all_cid[selected_cid]

print("x1_array shape:", x1_array.shape)  # (样本数, 基因数)

# 构造基因顺序表
gene_order_symbols = landmark_symbols + best_infer_symbols
print( len(landmark_symbols))
print(landmark_symbols)

# 检查对齐
print("是否匹配:", len(gene_order_symbols) == x1_array.shape[1])

# 保存对应表
df_gene_order = pd.DataFrame({
    "col_index": range(len(gene_order_symbols)),
    "gene_symbol": gene_order_symbols
})

x1_array shape: (1, 978)
978
['GNPDA1', 'CDH3', 'HDAC6', 'PARP2', 'MAMLD1', 'DNAJB6', 'SMC4', 'ABCC5', 'ABCB6', 'DNM1L', 'TSPAN3', 'KIF20A', 'ARL4C', 'TRAP1', 'G3BP1', 'MBNL2', 'CEBPZ', 'SLC25A13', 'CDK2', 'SORBS3', 'RBM6', 'CDK4', 'TXNDC9', 'ADAM10', 'TRIM13', 'CDK6', 'CDK7', 'TRIB1', 'MFSD10', 'SLC35B1', 'TIMM17B', 'CDKN1A', 'CDKN1B', 'AKAP8', 'STUB1', 'NET1', 'SMNDC1', 'CDKN2A', 'PAK4', 'TNIP1', 'IKZF1', 'TMEM5', 'HMG20B', 'MYL9', 'LYPLA1', 'PPIE', 'VAV3', 'LRRC41', 'CRTAP', 'VAT1', 'STK25', 'CEBPA', 'APPBP2', 'CEBPD', 'CHERP', 'HYOU1', 'RPP38', 'SLC35A1', 'DRAP1', 'PAICS', 'ST6GALNAC2', 'STAMBP', 'CENPE', 'NPRL2', 'IGF2BP2', 'YKT6', 'CGRRF1', 'RRAGA', 'GNB5', 'EBP', 'CNPY3', 'CETN3', 'YME1L1', 'TCFL5', 'KDM5B', 'POP4', 'ARPP19', 'ZNF274', 'MTHFD2', 'WASF3', 'UTP14A', 'FRS2', 'CLPX', 'PGRMC1', 'MALT1', 'CPSF4', 'BLCAP', 'TCERG1', 'RNPS1', 'TOMM34', 'PDIA5', 'MLLT11', 'EBNA1BP2', 'TMED10', 'ASCC3', 'SLC27A3', 'KIF2C', 'CCDC85B', 'TLK2', 'KDELR2', 'RAB31', 'B4GAT1', 'PAPD7', 'UBE2C', 

In [16]:
import pandas as pd

df_gene = pd.read_csv('../data/LINCS2020/geneinfo_processed.csv')

df_landmark_gene = df_gene[(df_gene['pr_is_bing'] == 1) & (df_gene['pr_is_lm'] == 1)]

lincs_order = df_landmark_gene["pr_gene_id"].tolist()
geneid2symbol = dict(zip(df_landmark_gene["pr_gene_id"], df_landmark_gene["pr_gene_symbol"]))


with open(os.path.join("/root/myproject/KnowledgeGraphEmbedding-master/data/DrugBank/S0", 'entities.dict')) as fin:
    entity2id = dict()
    for line in fin:
        eid, entity = line.strip().split('\t')
        entity2id[int(eid)] = entity

parsed = []
for eid, entity in entity2id.items():
    if entity.startswith("Gene::"):  # 只处理基因节点
        entrez_id = int(entity.split("::")[1])
        symbol = geneid2symbol.get(entrez_id, None)
        if symbol:  # 只保留 landmark
            parsed.append((eid, entity, entrez_id, symbol))
df_map = pd.DataFrame(parsed, columns=["dict_index", "hetionet_gene", "entrez_id", "gene_symbol"])

df_map["order"] = df_map["entrez_id"].apply(lambda x: lincs_order.index(x) if x in lincs_order else 1e9)
df_map = df_map.sort_values("order").drop(columns="order")


df_map.to_csv("hetionet_to_lincs_landmark_mapping.csv", index=False)
print("映射表已保存到 hetionet_to_lincs_landmark_mapping.csv")
print(df_map.head())

映射表已保存到 hetionet_to_lincs_landmark_mapping.csv
     dict_index hetionet_gene  entrez_id gene_symbol
677        9320   Gene::10007      10007      GNPDA1
757       10875    Gene::1001       1001        CDH3
316        4633   Gene::10013      10013       HDAC6
811       12201   Gene::10038      10038       PARP2
975       18958   Gene::10046      10046      MAMLD1
